In [1]:
import tensorflow as tf
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# fetch dataset 
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_diagnostic.data.features 
y = breast_cancer_wisconsin_diagnostic.data.targets 

In [2]:
import pandas as pd

# Data preprocessing
data=pd.concat([X,y],axis=1)
# if Malignant(M):1, if benign(B):0
data['Diagnosis']=data['Diagnosis'].apply(lambda val: 1 if val == 'M' else 0)
# Remove highly correlated features
correlation_mat = data.corr()
#correlation threshold
threshold = 0.95 #setting to 0.95

highly_correlated = correlation_mat[abs(correlation_mat) > threshold]
highly_correlated = highly_correlated[highly_correlated != 1].stack().index.tolist()

# which ones to remove
remove = set()
for (feature1, feature2) in highly_correlated:
    remove.add(feature1 if correlation_mat.loc[feature1, feature2] > 0 else feature2)

# drop highly correlated features from the dataframe
data = data.drop(remove, axis=1)


In [3]:
#import ydf
from sklearn.model_selection import KFold
import numpy as np
#####FOR WINDOWS####
#tuner=ydf.RandomSearchTuner(num_trials=50)
#tuner.choice("num_trees",[3,4,5,6,7,8,9,10])
#tuner.choice("max_depth",[3,4,5,6,7,8,9,10])
#kf=KFold(n_splits=9,shuffle=True, random_state=50)
#mod=[]
#eval=[]
#for traI, testI in kf.split(data):
   # train=data.iloc[traI]
   # test=data.iloc[testI]
   # model = ydf.RandomForestLearner(label='Diagnosis', num_trees=10,max_depth=5).train(train)
 #   mod.append(model)
#    evaluation=model.evaluate(test)
 #   eval.append(evaluation.accuracy)
#index=np.argmax(eval)
#mo=mod[index]
#mo.save("model")

In [8]:
from sklearn.model_selection import KFold
import numpy as np
import tensorflow_decision_forests as tfdf

#tuner=ydf.RandomSearchTuner(num_trials=50)
#tuner.choice("num_trees",[3,4,5,6,7,8,9,10])
#tuner.choice("max_depth",[3,4,5,6,7,8,9,10])
kf=KFold(n_splits=9,shuffle=True, random_state=50)
mod=[]
eval=[]
for traI, testI in kf.split(data):
    train=data.iloc[traI]
    test=data.iloc[testI]
    train= tfdf.keras.pd_dataframe_to_tf_dataset(train, label="Diagnosis")
    test= tfdf.keras.pd_dataframe_to_tf_dataset(test, label="Diagnosis")
    model=tfdf.keras.RandomForestModel(verbose=2)
    model.fit(train)
    model.compile(metrics=["accuracy"])
    evaluation = model.evaluate(test, return_dict=True)
    print()
    mod.append(model)
    eval.append(evaluation)



Use 8 thread(s) for training
Use /var/folders/4l/8w9kqmzs5kn2xsls939d3crr0000gn/T/tmpb_tzlwws as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'texture1': <tf.Tensor 'data:0' shape=(None,) dtype=float64>, 'smoothness1': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'compactness1': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'concavity1': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'concave_points1': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'symmetry1': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'fractal_dimension1': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'texture2': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'smoothness2': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'compactness2': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'concavity2': <tf.Tensor 'data_10:0' shape=(None,) dtype=float64>, 'concave_points2': <tf.Tensor 'data_11:0' shape=(None,) dtype=float64>

In [47]:
df=pd.DataFrame(eval)
temp=eval[0]['accuracy']
for x in range (0, len(eval)):
    if(temp<eval[x]["accuracy"]):
        temp=eval[x]['accuracy']
        index=x
bestmodel=mod[index]
bestmodel.save("my_saved_model")
print(eval[1]["accuracy"])
#mo=mod[index]


INFO:tensorflow:Assets written to: my_saved_model/assets


INFO:tensorflow:Assets written to: my_saved_model/assets


0.953125


In [48]:

bestmodel.save('/Users/mohammadmendahawi/BC/GUI/my_saved_model.h5')

/opt/anaconda3/envs/ml1/lib/python3.9/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


NotImplementedError: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.